<a href="https://colab.research.google.com/github/Chandan0731/bioinformatics_lab/blob/main/Experiment_7_Genome_Assembly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# 1. Install FastQC (Quality Checker) and Velvet (The Assembler)
print("Installing FastQC and Velvet...")
!apt-get install fastqc velvet > /dev/null
print("✅ Installation complete.")

# 2. Download the Raw Sequencing Data (FASTQ files)
print("Downloading raw sequencing reads...")
!wget -q https://zenodo.org/record/582600/files/mutant_R1.fastq
!wget -q https://zenodo.org/record/582600/files/mutant_R2.fastq

if os.path.exists("mutant_R1.fastq") and os.path.exists("mutant_R2.fastq"):
    print("✅ Files downloaded successfully.")
else:
    print("❌ Download failed. Please try again.")

Installing FastQC and Velvet...
Extracting templates from packages: 100%
✅ Installation complete.
✅ Files downloaded successfully.


In [2]:
print("Running FastQC on Read 1...")
!fastqc mutant_R1.fastq

print("Running FastQC on Read 2...")
!fastqc mutant_R2.fastq

# List the output files to confirm
print("\n--- Output Files ---")
!ls *.html
print("✅ Quality Check done. You can see the .html reports in the file browser.")

Running FastQC on Read 1...
Started analysis of mutant_R1.fastq
Approx 5% complete for mutant_R1.fastq
Approx 15% complete for mutant_R1.fastq
Approx 20% complete for mutant_R1.fastq
Approx 30% complete for mutant_R1.fastq
Approx 40% complete for mutant_R1.fastq
Approx 45% complete for mutant_R1.fastq
Approx 55% complete for mutant_R1.fastq
Approx 60% complete for mutant_R1.fastq
Approx 70% complete for mutant_R1.fastq
Approx 80% complete for mutant_R1.fastq
Approx 85% complete for mutant_R1.fastq
Approx 95% complete for mutant_R1.fastq
Analysis complete for mutant_R1.fastq
Running FastQC on Read 2...
Started analysis of mutant_R2.fastq
Approx 5% complete for mutant_R2.fastq
Approx 15% complete for mutant_R2.fastq
Approx 20% complete for mutant_R2.fastq
Approx 30% complete for mutant_R2.fastq
Approx 40% complete for mutant_R2.fastq
Approx 45% complete for mutant_R2.fastq
Approx 55% complete for mutant_R2.fastq
Approx 60% complete for mutant_R2.fastq
Approx 70% complete for mutant_R2.fa

In [3]:

import shutil

# 1. Create a clean directory for the output
if os.path.exists("assembly_output"):
    shutil.rmtree("assembly_output")
os.makedirs("assembly_output")

# 2. Run 'velveth' (Hashing)
# This builds a "Hash Table" (index) of all the k-mers (word fragments)
print("Step 1: Hashing reads (velveth)...")
!velveth assembly_output 31 -short -separate -fastq mutant_R1.fastq mutant_R2.fastq

# 3. Run 'velvetg' (Graph Building)
# This connects the fragments into a De Bruijn Graph to build the contigs
print("Step 2: Building graph (velvetg)...")
!velvetg assembly_output -cov_cutoff auto

# 4. View Results
print("\n--- ASSEMBLY COMPLETED ---")
result_path = "assembly_output/contigs.fa"

if os.path.exists(result_path):
    print(f"✅ Assembly successful! Output saved to: {result_path}")

    # Show the top 10 lines of the result
    print("\nFirst 10 assembled sequences (Contigs):")
    with open(result_path, 'r') as f:
        print(f.read(500)) # Print first 500 characters
else:
    print("❌ Assembly failed.")

Step 1: Hashing reads (velveth)...
[0.000000] Reading FastQ file mutant_R1.fastq;
[0.056853] 12480 sequences found
[0.056876] Done
[0.056929] Reading FastQ file mutant_R2.fastq;
[0.111949] 12480 sequences found
[0.111973] Done
[0.112068] Reading read set file assembly_output/Sequences;
[0.117135] 24960 sequences found
[0.142154] Done
[0.142186] 24960 sequences in total.
[0.142427] Writing into roadmap file assembly_output/Roadmaps...
[0.188147] Inputting sequences...
[0.188182] Inputting sequence 0 / 24960
[0.983871]  === Sequences loaded in 0.795727 s
[0.983915] Done inputting sequences
[0.983918] Destroying splay table
[0.995050] Splay table destroyed
Step 2: Building graph (velvetg)...
[0.000000] Reading roadmap file assembly_output/Roadmaps
[0.045449] 24960 roadmaps read
[0.045676] Creating insertion markers
[0.048965] Ordering insertion markers
[0.067781] Counting preNodes
[0.071358] 64223 preNodes counted, creating them now
[0.172092] Adjusting marker info...
[0.175812] Connectin

In [5]:
# Block 4: Assembly Statistics and Validation
# We calculate metrics (Contig count, N50, Max Length) to prove the assembly worked.

# 1. Install Biopython (Fixes 'No module named Bio' error)
!pip install biopython

from Bio import SeqIO
import numpy as np
import os

contig_file = "assembly_output/contigs.fa"

# Check if file exists before analyzing
if not os.path.exists(contig_file):
    print(f"❌ Error: The file '{contig_file}' was not found.")
    print("Please make sure you ran Block 3 successfully first.")
else:
    print(f"\nAnalyzing Assembly Results from: {contig_file}\n")

    # 2. Read the contigs
    contigs = list(SeqIO.parse(contig_file, "fasta"))
    lengths = [len(c.seq) for c in contigs]
    lengths.sort(reverse=True) # Sort from biggest to smallest

    if len(lengths) > 0:
        # 3. Calculate Statistics
        total_bases = sum(lengths)
        num_contigs = len(lengths)
        max_len = lengths[0]
        min_len = lengths[-1]

        # Calculate N50 (Standard assembly quality metric)
        cumsum = np.cumsum(lengths)
        n50_idx = np.searchsorted(cumsum, total_bases / 2)
        n50 = lengths[n50_idx]

        # 4. Print the "Report Card"
        print("="*40)
        print("      GENOME ASSEMBLY REPORT")
        print("="*40)
        print(f"Total Contigs Found:   {num_contigs}")
        print(f"Total Genome Length:   {total_bases} bp")
        print(f"Largest Contig:        {max_len} bp")
        print(f"Smallest Contig:       {min_len} bp")
        print(f"N50 Score:             {n50}")
        print("="*40)

        # 5. Show the actual Sequences (Evidence)
        print("\nTop 5 Longest Assembled Sequences:")
        for i in range(min(5, len(contigs))):
            print(f"\n>Contig_{i+1} (Length: {len(contigs[i].seq)} bp)")
            # Print first 100 bases only
            print(str(contigs[i].seq)[:100] + "...")

    else:
        print("❌ valid contigs file found, but it is empty.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 74.5 MB/s eta 0:00:00

Analyzing Assembly Results from: assembly_output/contigs.fa

      GENOME ASSEMBLY REPORT
Total Contigs Found:   20
Total Genome Length:   179090 bp
Largest Contig:        36563 bp
Smallest Contig:       61 bp
N50 Score:             30380

Top 5 Longest Assembled Sequences:

>Contig_1 (Length: 36563 bp)
CAATGGTTTCATCCAAAGCTTAAAAGATGATAAAGCAGAAGGTCCCTATGTAGCAATTTCTAAAGTTGGAAAAGGTAAAGCAGCATTTATCGGTGATTCA...

>Contig_2 (Length: 790 bp)
GGTTCTGTTGCAAAGTAAAAAAATATAGCTAACCACTAATTTATCATGTCAGTGTTCGCTTAACTTGCTAGCATGATGCTAATTTCGTGGCATGGCGAAA...

>Contig_3 (Length: 34959 bp)
ATACTATAAATTCAACTTTGCAACAGAACCTTCTACATTAAATACTTTGTCAATGAGATCATCTACATCTTTAAATTTAGAATAATTTGCATATGGATCT...

>Contig_4 (Length: 30380 bp)
AAAGCAATCCTAAGTAAAATTGCAGATAAGGGGTACAGAAAAACTAGACTTGATTACAAAATGGAGCTTGGGACATAAATGATTTTTTAAAAATGAGATG...

>Contig_5 (Length: 17420 bp)
GGCTGGTTTTATTGGGTCACATTTAGTAGATGATTTGCAACAAGATTATGATGTTTATGTTCTAGATAACTATAGAACAGGTA